In [1]:
import pandas as pd
import sqlite3 as sq
import matplotlib as mpl
from matplotlib import rcParams
import matplotlib.pyplot as plt
import numpy as np
import requests
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', 150)
np.set_printoptions(suppress=True)

In [2]:
pep = pd.read_csv('../data/2023_CountyPEP.csv')

In [3]:
pep.head()

,NAME,2020 Decennial,2020,2021,2022,2023
0,Tennessee,6910786,6926091,6963709,7048976,7126489
1,"Anderson County, Tennessee",77147,77320,77615,78929,80234
2,"Bedford County, Tennessee",50237,50385,51184,51964,53055
3,"Benton County, Tennessee",15867,15855,15837,15994,16103
4,"Bledsoe County, Tennessee",14917,14956,14837,14824,15060


In [4]:
pep = pep.rename(columns = {'2023': '2023 Census'})

In [5]:
pep.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96 entries, 0 to 95
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   NAME            96 non-null     object
 1   2020 Decennial  96 non-null     int64 
 2   2020            96 non-null     int64 
 3   2021            96 non-null     int64 
 4   2022            96 non-null     int64 
 5   2023 Census     96 non-null     int64 
dtypes: int64(5), object(1)
memory usage: 4.6+ KB


In [6]:
conn = sq.connect('../../Data-Pipelines/Outputs/WoodsandPooleandAffiliated.db')
sql_query = pd.read_sql('SELECT * FROM [WPPopulations_2023Base]', conn)
pop = pd.DataFrame(sql_query)
pop = pop[['NAME', 'Year', 'totalpop']]
pop = pop.loc[pop['Year'] != 'None']

In [7]:
pop.head(2)

,NAME,Year,totalpop
0,"Cheatham County, Tennessee",1990,27310.0
1,"Cheatham County, Tennessee",1991,27850.0


In [8]:
pop['Year'] = pop['Year'].astype(int)

In [9]:
pop.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1159 entries, 0 to 34098
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   NAME      1159 non-null   object 
 1   Year      1159 non-null   int32  
 2   totalpop  1159 non-null   float64
dtypes: float64(1), int32(1), object(1)
memory usage: 31.7+ KB


In [10]:
pop = pop[pop['Year'] > 2022]

In [11]:
pop['Year'] = pop['Year'].astype(str)

In [12]:
wide_df = pop.pivot_table(index='NAME', columns='Year', values='totalpop')

In [13]:
wide_df = wide_df.reset_index().rename_axis(None, axis=1)

In [14]:
wide_df.head(2)

,NAME,2023,2024,2025,2026,2027,2028,2029,2030,2031,2032,2033,2034,2035,2036,2037,2038,2039,2040,2041,2042,2043,2044,2045,2046,2047,2048,2049,2050
0,"Cheatham County, Tennessee",42068.0,42306.0,42546.0,42787.0,43030.0,43273.0,43519.0,43766.0,44014.0,44264.0,44515.0,44767.0,45021.0,45276.0,45533.0,45791.0,46051.0,46312.0,46574.0,46838.0,47104.0,47371.0,47639.0,47909.0,48181.0,48454.0,48729.0,49005.0
1,"Davidson County, Tennessee",712937.0,717674.0,722340.0,726916.0,731391.0,735759.0,740023.0,744193.0,748264.0,752215.0,756059.0,759777.0,763361.0,766798.0,770090.0,773262.0,776324.0,779297.0,782199.0,785039.0,787833.0,790590.0,793331.0,796062.0,798795.0,801550.0,804332.0,807153.0


In [15]:
wide_df = wide_df.rename(columns = {'2023': '2023 WP'})

In [16]:
merged_df = pd.merge(wide_df, pep[['NAME', '2023 Census']], on = 'NAME', how = 'inner')
column_order = ['NAME', '2023 Census'] + [col for col in merged_df.columns if col not in ['NAME', '2023 Census']]
merged_df = merged_df[column_order]

In [17]:
merged_df.head(3)

,NAME,2023 Census,2023 WP,2024,2025,2026,2027,2028,2029,2030,2031,2032,2033,2034,2035,2036,2037,2038,2039,2040,2041,2042,2043,2044,2045,2046,2047,2048,2049,2050
0,"Cheatham County, Tennessee",42254,42068.0,42306.0,42546.0,42787.0,43030.0,43273.0,43519.0,43766.0,44014.0,44264.0,44515.0,44767.0,45021.0,45276.0,45533.0,45791.0,46051.0,46312.0,46574.0,46838.0,47104.0,47371.0,47639.0,47909.0,48181.0,48454.0,48729.0,49005.0
1,"Davidson County, Tennessee",712334,712937.0,717674.0,722340.0,726916.0,731391.0,735759.0,740023.0,744193.0,748264.0,752215.0,756059.0,759777.0,763361.0,766798.0,770090.0,773262.0,776324.0,779297.0,782199.0,785039.0,787833.0,790590.0,793331.0,796062.0,798795.0,801550.0,804332.0,807153.0
2,"Dickson County, Tennessee",56729,56221.0,56677.0,57129.0,57576.0,58016.0,58447.0,58872.0,59291.0,59703.0,60106.0,60502.0,60888.0,61265.0,61631.0,61987.0,62333.0,62672.0,63005.0,63332.0,63655.0,63975.0,64294.0,64611.0,64929.0,65247.0,65568.0,65893.0,66220.0


In [18]:
non_adj = merged_df
non_adj = non_adj.drop(columns = ['2023 Census', '2023 WP']).set_index('NAME')
non_adj = non_adj.add_suffix(" Non-Adjusted").reset_index()

In [19]:
non_adj.head()

,NAME,2024 Non-Adjusted,2025 Non-Adjusted,2026 Non-Adjusted,2027 Non-Adjusted,2028 Non-Adjusted,2029 Non-Adjusted,2030 Non-Adjusted,2031 Non-Adjusted,2032 Non-Adjusted,2033 Non-Adjusted,2034 Non-Adjusted,2035 Non-Adjusted,2036 Non-Adjusted,2037 Non-Adjusted,2038 Non-Adjusted,2039 Non-Adjusted,2040 Non-Adjusted,2041 Non-Adjusted,2042 Non-Adjusted,2043 Non-Adjusted,2044 Non-Adjusted,2045 Non-Adjusted,2046 Non-Adjusted,2047 Non-Adjusted,2048 Non-Adjusted,2049 Non-Adjusted,2050 Non-Adjusted
0,"Cheatham County, Tennessee",42306.0,42546.0,42787.0,43030.0,43273.0,43519.0,43766.0,44014.0,44264.0,44515.0,44767.0,45021.0,45276.0,45533.0,45791.0,46051.0,46312.0,46574.0,46838.0,47104.0,47371.0,47639.0,47909.0,48181.0,48454.0,48729.0,49005.0
1,"Davidson County, Tennessee",717674.0,722340.0,726916.0,731391.0,735759.0,740023.0,744193.0,748264.0,752215.0,756059.0,759777.0,763361.0,766798.0,770090.0,773262.0,776324.0,779297.0,782199.0,785039.0,787833.0,790590.0,793331.0,796062.0,798795.0,801550.0,804332.0,807153.0
2,"Dickson County, Tennessee",56677.0,57129.0,57576.0,58016.0,58447.0,58872.0,59291.0,59703.0,60106.0,60502.0,60888.0,61265.0,61631.0,61987.0,62333.0,62672.0,63005.0,63332.0,63655.0,63975.0,64294.0,64611.0,64929.0,65247.0,65568.0,65893.0,66220.0
3,"Houston County, Tennessee",8244.0,8257.0,8269.0,8282.0,8295.0,8308.0,8320.0,8333.0,8346.0,8359.0,8372.0,8383.0,8396.0,8409.0,8422.0,8435.0,8448.0,8461.0,8474.0,8487.0,8500.0,8513.0,8526.0,8539.0,8552.0,8565.0,8579.0
4,"Humphreys County, Tennessee",19171.0,19204.0,19238.0,19271.0,19303.0,19337.0,19370.0,19403.0,19436.0,19470.0,19504.0,19537.0,19570.0,19604.0,19638.0,19671.0,19705.0,19739.0,19773.0,19807.0,19841.0,19875.0,19909.0,19943.0,19978.0,20012.0,20046.0


In [20]:
years = list(merged_df.columns)
years.remove('NAME')
years.remove('2023 Census')
years.remove('2023 WP')
years.remove('2050')

In [21]:
def apply_formula(row):
    # Assuming '2023' is the base year for census and WP, and '2050' is the last year
    base_year = 2023
    final_year = 2050

    # Calculate the denominator part of the formula
    denominator = final_year - base_year
    
    # If denominator is zero, return 0 to avoid division by zero error
    if denominator == 0:
        return 0
    # Loop through the years between 2024 and 2049 (inclusive)
    for year in range(base_year + 1, final_year):
        # Calculate the adjusted value for the current year
        adjusted_value = row[f'{year}'] - (row[f'{base_year} WP'] - row[f'{base_year} Census']) / denominator * (final_year - int(year))
       # Update the value for the current year
        row[f'{year}'] = adjusted_value
    
    return row

# Apply the formula to each row in the DataFrame
merged_df = merged_df.apply(apply_formula, axis=1)

In [22]:
merged_df.head()

,NAME,2023 Census,2023 WP,2024,2025,2026,2027,2028,2029,2030,2031,2032,2033,2034,2035,2036,2037,2038,2039,2040,2041,2042,2043,2044,2045,2046,2047,2048,2049,2050
0,"Cheatham County, Tennessee",42254,42068.0,42485.111111,42718.222222,42952.333333,43188.444444,43424.555556,43663.666667,43903.777778,44144.888889,44388.000000,44632.111111,44877.222222,45124.333333,45372.444444,45622.555556,45873.666667,46126.777778,46380.888889,46636.000000,46893.111111,47152.222222,47412.333333,47673.444444,47936.555556,48201.666667,48467.777778,48735.888889,49005.0
1,"Davidson County, Tennessee",712334,712937.0,717093.333333,721781.666667,726380.000000,730877.333333,735267.666667,739554.000000,743746.333333,747839.666667,751813.000000,755679.333333,759419.666667,763026.000000,766485.333333,769799.666667,772994.000000,776078.333333,779073.666667,781998.000000,784860.333333,787676.666667,790456.000000,793219.333333,795972.666667,798728.000000,801505.333333,804309.666667,807153.0
2,"Dickson County, Tennessee",56729,56221.0,57166.185185,57599.370370,58027.555556,58448.740741,58860.925926,59267.111111,59667.296296,60060.481481,60444.666667,60821.851852,61189.037037,61547.222222,61894.407407,62231.592593,62558.777778,62878.962963,63193.148148,63501.333333,63805.518519,64106.703704,64406.888889,64705.074074,65004.259259,65303.444444,65605.629630,65911.814815,66220.0
3,"Houston County, Tennessee",8393,8231.0,8400.000000,8407.000000,8413.000000,8420.000000,8427.000000,8434.000000,8440.000000,8447.000000,8454.000000,8461.000000,8468.000000,8473.000000,8480.000000,8487.000000,8494.000000,8501.000000,8508.000000,8515.000000,8522.000000,8529.000000,8536.000000,8543.000000,8550.000000,8557.000000,8564.000000,8571.000000,8579.0
4,"Humphreys County, Tennessee",19209,19139.0,19238.407407,19268.814815,19300.222222,19330.629630,19360.037037,19391.444444,19421.851852,19452.259259,19482.666667,19514.074074,19545.481481,19575.888889,19606.296296,19637.703704,19669.111111,19699.518519,19730.925926,19762.333333,19793.740741,19825.148148,19856.555556,19887.962963,19919.370370,19950.777778,19983.185185,20014.592593,20046.0


In [23]:
data = merged_df.merge(non_adj, on = 'NAME')

In [24]:
data.to_csv('../data/smoothedtotalpop_2023PEP.csv', index = False)

In [27]:
#export to the SQLite database
conn = sq.connect('../../Data-Pipelines/Outputs/WoodsandPooleandAffiliated.db')
data.to_sql('WP_Population_2023Base_Smoothed2023PEP', conn, if_exists = 'replace', index = False)

15